# Labo 5 : Vector Embedding

> Authors : Stefani Massimo, Häffner Edwin

## Partie 1

In [1]:
# Installation de gensim et scipy en dernière version
#!pip install gensim scipy

In [1]:
#Installation du modèle entraîné sur Google News 
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [2]:
#Chargement du modèle
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format("~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz", binary=True)


### Réponse aux questions 

>a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

Sur mon OS Linux, je vois que le notebook prends 4.57 GiB en mémoire. Visual studio code en prends 8.52 GiB mais je ne pense pas que c'est lié ! 



> b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?


In [3]:
print(f"La dimension de l'espace vectoriel est de {w2v_vectors.vector_size} dimensions")


La dimension de l'espace vectoriel est de 300 dimensions


> c. Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher cinq mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.


In [4]:
print(f"La taille du vocabulaire est de {len(w2v_vectors)} mots et voici des mots qui sont dans le vocabulaire (ou pas): ")

for word in ["natural", "language", "processing", "Yverdon", "school", "bumfuzzle", "taradiddle", "collywobbles", "Yverdon-Les-Bains", "Puidoux"]:
    if word in w2v_vectors :
        print(f"Connu : {word}")
    else :
        print(f"Inconnu : {word}")

        


La taille du vocabulaire est de 3000000 mots et voici des mots qui sont dans le vocabulaire (ou pas): 
Connu : natural
Connu : language
Connu : processing
Connu : Yverdon
Connu : school
Inconnu : bumfuzzle
Inconnu : taradiddle
Connu : collywobbles
Inconnu : Yverdon-Les-Bains
Inconnu : Puidoux


> d. Quelle est la similarité entre les mots rabbit et carrot ? Veuillez rappeler comment on mesure les similarités entre deux mots grâce à leurs vecteurs.

La similarité entre deux mots est généralement mesurée en calculant la similarité cosinus entre leurs vecteurs. En gros on calcule le cosinus de l'angle entre les deux vecteurs puis si les deux vecteurs sont identiques, la similarité est de 1. Si les vecteurs sont orthogonaux, la similarité est de 0. Si les vecteurs sont opposés, la similarité est de -1.

In [ ]:
print('La similarité entre \"rabbit\" et \"carrot\" est de : %.4f' % (w2v_vectors.similarity("rabbit", "carrot")))



La similarité entre "rabbit" et "carrot" est de : 0.3631


> e. onsidérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la similarité entre les deux mots. Veuillez indiquer si les similarités obtenues correspondent à vos intuitions sur la proximité des sens des mots.


In [6]:
pairs = [
    ('rabbit', 'animal'), # Un lapin EST un animal
    ('rabbit', 'hare'), # Le lièvre est un genre de lapin "sauvage"
    ('rabbit', 'bunny'),  # Version cute de lapin
    ('rabbit', 'easter'),    # lapin de Pâques
    ('rabbit', 'vertex'), #Rien à voir
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, w2v_vectors.similarity(w1, w2)))

'rabbit'	'animal'	0.53
'rabbit'	'hare'	0.61
'rabbit'	'bunny'	0.64
'rabbit'	'easter'	0.19
'rabbit'	'vertex'	0.07


> Question e : Discussion

- On voit que lapin et animal est assez similaire ce qui fais sens
- Lapin et lièvre sont encore plus similaire ce qui fais aussi sens
- Un peu bizarre que rabbit et bunny ne soient pas plus similaire que ça vu que c'est littéralement le même animal, juste que bunny est la forme plus colloquial. 
- Rabbit et easter ne sont pas similaire ce qui fait sens, mais il y a quand même un lien (lapin de Pâques donc la similitude n'est pas si basse)
- Enfin rabbit et vertex (sommet) n'ont vraiment rien à voir donc 0.07 n'est pas surprenant.

> f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ?

In [ ]:
w1 = "awful"
w2 = "good"
print(f'Similarité entre {w1} et {w2} %.4f' % (w2v_vectors.similarity(w1, w2)))

Similarité entre awful et good 0.4928


> Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

Je pense que awful et good peuvent être utilisé dans des même contexte, comme lorsqu'on a un article qui évalue des choses, on peut à la fois considérer quelque chose de bon ou de mauvais. (Ex : The movie was awful but the acting was good). 



> g. En vous aidant de la documentation de Gensim sur KeyedVectors, obtenez les scores du modèle word2vec sur les données de test WordSimilarity-353. Veuillez rappeler en 1-2 phrases comment les différents scores sont calculés.

- Le score de Pearson est une mesure de la "force" de la relation entre deux variables, leurs associations l'une avec l'autre. C'est une mesure qui est linéaire et qui varie entre -1 et 1. Ici, on compare les similarités entre les mots du modèle et les similarités entre les mots dans le dataset, donc les similarités jugées par des êtres humains. Le coefficient de Pearson est calculé en divisant la covariance des deux variables par le produit de leurs écarts-types. (La formule est sur le wikipédia si vous voulez : https://en.wikipedia.org/wiki/Pearson_correlation_coefficient). Ici on a un score de 0.62 ce qui indique une bonne corrélation entre les résultats du modèle et les jugements humains.

- Ensuite, le score de Spearman permet de mesurer cette même force de relation mais de manière non linéaire. Il est aussi entre -1 et 1. Le coefficient de Spearman est calculé en convertissant les données en rangs, puis en appliquant la formule de Pearson sur ces rangs. Les rangs sont les positions des valeurs dans l'ordre croissant btw. Ici on a un score de 0.66 ce qui est encore mieux ce qui fait sens car utiliser les rangs permet de mieux capturer les relations non linéaires entre les variables !

In [45]:
from gensim.test.utils import datapath

def calculate_score_pearson_spearman_wordsim353(wv):
    
    pearson_result, significance_result, _ = wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

    print("Résultat de Pearson :")
    print(f"  Statistique : {pearson_result.statistic}")
    print(f"  Valeur P : {pearson_result.pvalue}")

    print("\nRésultat de Significativité (Spearman) :")
    print(f"  Statistique : {significance_result.statistic}")
    print(f"  Valeur P : {significance_result.pvalue}")

calculate_score_pearson_spearman_wordsim353(w2v_vectors)



Résultat de Pearson :
  Statistique : 0.6238773472434951
  Valeur P : 1.7963233960134136e-39

Résultat de Significativité (Spearman) :
  Statistique : 0.6589215888009288
  Valeur P : 2.5346056459149263e-45


>h. En vous aidant de la documentation, calculez le score du modèle word2vec sur les données
questions-words.txt. Attention, cette évaluation prend une dizaine de minutes, donc il vaut
mieux commencer par tester avec un fragment de ce fichier (copier/coller les 100 premières
analogies). Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [9]:
word_analogies = w2v_vectors.evaluate_word_analogies("./question-words-small.txt") 

print(word_analogies)

(0.81, [{'section': 'capital-common-countries', 'correct': [('ATHENS', 'GREECE', 'BANGKOK', 'THAILAND'), ('ATHENS', 'GREECE', 'BEIJING', 'CHINA'), ('ATHENS', 'GREECE', 'BERLIN', 'GERMANY'), ('ATHENS', 'GREECE', 'BERN', 'SWITZERLAND'), ('ATHENS', 'GREECE', 'CAIRO', 'EGYPT'), ('ATHENS', 'GREECE', 'CANBERRA', 'AUSTRALIA'), ('ATHENS', 'GREECE', 'HAVANA', 'CUBA'), ('ATHENS', 'GREECE', 'HELSINKI', 'FINLAND'), ('ATHENS', 'GREECE', 'ISLAMABAD', 'PAKISTAN'), ('ATHENS', 'GREECE', 'MADRID', 'SPAIN'), ('ATHENS', 'GREECE', 'MOSCOW', 'RUSSIA'), ('ATHENS', 'GREECE', 'OSLO', 'NORWAY'), ('ATHENS', 'GREECE', 'OTTAWA', 'CANADA'), ('ATHENS', 'GREECE', 'PARIS', 'FRANCE'), ('ATHENS', 'GREECE', 'ROME', 'ITALY'), ('ATHENS', 'GREECE', 'STOCKHOLM', 'SWEDEN'), ('ATHENS', 'GREECE', 'TEHRAN', 'IRAN'), ('ATHENS', 'GREECE', 'TOKYO', 'JAPAN'), ('BAGHDAD', 'IRAQ', 'BANGKOK', 'THAILAND'), ('BAGHDAD', 'IRAQ', 'BEIJING', 'CHINA'), ('BAGHDAD', 'IRAQ', 'BERLIN', 'GERMANY'), ('BAGHDAD', 'IRAQ', 'CAIRO', 'EGYPT'), ('BAGHDAD'

In [32]:
# Avec seulement les cents première lignes du dataset, on trouves ces résultats :
# (word_analogies est un tuple où le premier élément est la précision et le second élément contient les détails des sections)

def print_word_analogies_data(analogies_output):

	accuracy = analogies_output[0]
	section_details = analogies_output[1]

	total_correct = 0
	total_incorrect = 0

	# On va calculer le nombre total d'analogies correctes et incorrectes
	for section in section_details:
		if 'correct' in section and 'incorrect' in section:
			total_correct += len(section['correct'])
			total_incorrect += len(section['incorrect'])

	total_analogies = total_correct + total_incorrect

	print(f"Nombre d'analogies : {total_analogies}") # On vérifie le nombre total d'analogie, devrait être 200 car il y a 2 analogies par ligne
	print(f"Nombre d'analogies correctes : {total_correct}")
	print(f"Nombre d'analogies incorrectes : {total_incorrect}")
	print(f"Précision : {accuracy:.2f}")

print_word_analogies_data(word_analogies)


Nombre d'analogies : 200
Nombre d'analogies correctes : 162
Nombre d'analogies incorrectes : 38
Précision : 0.81


In [35]:
# Ensuite nous allons évaluer le modèle sur le jeu total de données
word_analogies = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
print_word_analogies_data(word_analogies)

Nombre d'analogies : 38660
Nombre d'analogies correctes : 28614
Nombre d'analogies incorrectes : 10046
Précision : 0.74


Sur mon super pc trop bien, ça a pris 3m35 pour le calcul !

Mais bref, le score est calculé de façon très simple. Dans une analogie, on a 4 mots A, B, C et D. On veut savoir si A est à B ce que C est à D. Donc on cherche ce mot D en faisant vector(A) - vector(B) + vector(C). 
Si le mot D est en effet le mot qui est indiqué dans le fichier, alors on a une analogie correcte. On fait ça pour toutes les analogies du fichier et on divise le nombre d'analogies correctes par le nombre total d'analogies.

On voit aussi que sur le corpus complet on a un score moins élevé que sur le corpus réduit. Ça fait sens car le corpus complet est plus grand et donc il y a plus de chances d'avoir des mots qui ne sont pas dans le vocabulaire du modèle. Mais ça aurait aussi pu être l'inverse, ça dépend de la chance qu'on a sur le corpus réduit.

Mais on voit tout de même que le corpus réduit permet d'avoir des valeurs assez proches de celles du corpus complet, donc ça fait sens de l'utiliser pour faire des tests rapides plus tard dans le labo.

## Partie 2

a. En utilisant gensim.downloader (voir question 1) récupérez le corpus qui contient les 108 pre-
miers caractères de Wikipédia (en anglais) avec la commande : corpus = api.load('text8').
Combien de phrases et de mots (tokens) possède ce corpus ?

In [10]:
corpus = api.load('text8')


[==================================================] 100.0% 31.6/31.6MB downloaded


In [11]:
# Calcul du nombre de phrases et de mots (tokens) dans le corpus
num_sentences = sum(1 for _ in corpus)
num_tokens = sum(len(sentence) for sentence in corpus)

print(f"Le corpus contient {num_sentences} phrases et {num_tokens} mots (tokens).")

Le corpus contient 1701 phrases et 17005207 mots (tokens).


> b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus (voir la documentation de
Word2vec). Si nécessaire, procédez progressivement, en commençant par utiliser 1% du corpus,
puis 10%, etc., pour contrôler le temps nécessaire.

> • Veuillez indiquer la dimension choisie pour le embedding de ce nouveau modèle.
> • Combien de temps prend l’entraînement sur le corpus total ?
> • Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [12]:
from gensim.models import Word2Vec

def train_word2vec_model(corpus, num_sentences, vector_size=100, window=5, min_count=5, workers=4, percent=1):
    """
    Entraîne un modèle Word2Vec sur le corpus donné.
    
    :param corpus: Corpus d'entraînement
    :param num_sentences: Nombre total de phrases dans le corpus
    :param vector_size: Dimension des vecteurs de mots
    :param window: Taille de la fenêtre contextuelle
    :param min_count: Fréquence minimale des mots pour être inclus dans le vocabulaire
    :param workers: Nombre de threads à utiliser pour l'entraînement
    :param percent: Pourcentage de phrases à utiliser pour l'entraînement. Default = 1 (100%)
    :return: Modèle Word2Vec entraîné
    """
    # Limiter le corpus à un certain pourcentage
    num_sentences_limited = int(num_sentences * percent)
    limited_corpus = [sentence for i, sentence in enumerate(corpus) if i < num_sentences_limited]    
    # Entraîner le modèle Word2Vec
    model = Word2Vec(sentences=limited_corpus, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    
    return model

In [ ]:
model_1_percent = train_word2vec_model(corpus, num_sentences, percent=0.01)
print("Modèle entraîné sur 1% du corpus.")
## Pour 1% il a mis 0.9 secondes

Modèle entraîné sur 1% du corpus.


In [14]:
model_1_percent.save("word2vec_model_1_percent.model")

In [ ]:
model_10_percent = train_word2vec_model(corpus, num_sentences, percent=0.1)
print("Modèle entraîné sur 10% du corpus.")
# Pour 10% il a mis 2.2 secondes.

Modèle entraîné sur 10% du corpus.


In [16]:
model_10_percent.save("word2vec_model_10_percent.model")

In [ ]:
model_50_percent = train_word2vec_model(corpus, num_sentences, percent=0.5)
print("Modèle entraîné sur 50% du corpus.")
# Pour 50% il a mis 9.4 secondes.

Modèle entraîné sur 50% du corpus.


In [18]:
model_50_percent.save("word2vec_model_50_percent.model")

In [ ]:
model_100_percent = train_word2vec_model(corpus, num_sentences)
print("Modèle entraîné sur 100% du corpus.")
# Pour 100% il a mis 20.1 secondes.

Modèle entraîné sur 100% du corpus.


In [20]:
model_100_percent.save("word2vec_model_100_percent.model")

In [ ]:
## Taille du modèle
import os
model_1_percent_size = os.path.getsize("word2vec_model_1_percent.model")
model_10_percent_size = os.path.getsize("word2vec_model_10_percent.model")
model_50_percent_size = os.path.getsize("word2vec_model_50_percent.model")
model_100_percent_size = os.path.getsize("word2vec_model_100_percent.model")
print(f"Taille du modèle 1% : {model_1_percent_size / (1024 * 1024):.2f} Mo")
print(f"Taille du modèle 10% : {model_10_percent_size / (1024 * 1024):.2f} Mo")
print(f"Taille du modèle 50% : {model_50_percent_size / (1024 * 1024):.2f} Mo")
print(f"Taille du modèle 100% : {model_100_percent_size / (1024 * 1024):.2f} Mo")

Taille du modèle 1% : 3.18 Mo
Taille du modèle 10% : 15.41 Mo
Taille du modèle 50% : 38.64 Mo
Taille du modèle 100% : 56.47 Mo


71290

> c. Mesurez la qualité de ce modèle comme en (1g) et (1h). Ce modèle est-il meilleur que celui
entraîné sur Google News ? Quelle est selon vous la raison de la différence ?

In [50]:
model_100_percent.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

# Affichage des résultats de Pearson et Spearman
calculate_score_pearson_spearman_wordsim353(model_100_percent.wv)
# Affichage de la similarité entre "rabbit" et "carrot" pour le modèle entraîné sur 100% du corpus

print("\nEnsuite voici les résultat sur les analogies :")
# Evaluate word analogies
word_analogies_small = model_100_percent.wv.evaluate_word_analogies("./question-words-small.txt")

print_word_analogies_data(word_analogies_small)


Résultat de Pearson :
  Statistique : 0.6182272608413365
  Valeur P : 2.184957571285799e-38

Résultat de Significativité (Spearman) :
  Statistique : 0.6290602100641124
  Valeur P : 4.530351744300017e-40

Ensuite voici les résultat sur les analogies :
Nombre d'analogies : 200
Nombre d'analogies correctes : 64
Nombre d'analogies incorrectes : 136
Précision : 0.32


On voit ici que le modèle est légèrement moins bon que le modèle de google news ! Mais cela reste tout de même très bon pour un modèle entraîné chez nous pour Pearson et Spearman.

Par contre, pour les analogies le modèle de google news est bien bien meilleurs (0.74 de précision contre 0.32). Je pense que c'est parceque le modèle de google news est simplement beaucoup plus grand et donc il a plus de chances d'avoir vu les mots dans le corpus. Le modèle de google news a été aussi entraîné sur un corpus plus diversifié (actualités) avec une dimension vectorielle plus grande (300 vs 100), ce qui lui permet de surement capturer des nuances sémantiques plus fines et des relations analogiques plus complexes.

> d. Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus
text8 et des dépêches économiques de Reuters fourni par l’enseignant et appelé
wikipedia_augmented.zip (à décompresser en un fichier ‘.dat’ de 413 Mo). Entraînez un
nouveau modèle word2vec sur ce corpus, en précisant la dimension choisie pour les embeddings.

In [ ]:
from gensim.models.word2vec import Text8Corpus

new_corpus_path = "./wikipedia_augmented.dat"

corpus_text8 = Text8Corpus(new_corpus_path)
    
sentences_list = list(corpus_text8)
num_sentences_new = len(sentences_list)
num_tokens_new = sum(len(sentence) for sentence in sentences_list)
    

print(f"Le nouveau corpus contient {num_sentences_new} phrases et {num_tokens_new} mots (tokens).")

Le nouveau corpus contient 7011 phrases et 70102071 mots (tokens).


In [ ]:
import time

start_time = time.time()
# Entraînement d'un nouveau modèle Word2Vec sur le corpus étendu
new_model = train_word2vec_model(corpus_text8, num_sentences_new, percent=1)
training_time = time.time() - start_time

print(f"Temps d'entraînement: {training_time:.2f} secondes ({training_time/60:.2f} minutes)")
# Sauvegarde du modèle
new_model.save("word2vec_new_corpus.model")
print("Modèle entraîné et sauvegardé sur le corpus étendu.")

Temps d'entraînement: 92.32 secondes (1.54 minutes)
Modèle entraîné et sauvegardé sur le corpus étendu.


In [ ]:
# Calcul de la taille du modèle
import os
model_size_mb = os.path.getsize("word2vec_new_corpus.model") / (1024 * 1024)
print(f"Taille du modèle: {model_size_mb:.2f} Mo")


Taille du modèle: 3.71 Mo


124372

> e. Testez ce modèle comme en (1g) et (1h). Est-il meilleur que le précédent ? 

In [52]:
# Verification de peasrson et spearman
calculate_score_pearson_spearman_wordsim353(new_model.wv)

print("\nEnsuite voici les résultat sur les analogies :")
word_analogies_new = new_model.wv.evaluate_word_analogies("./question-words-small.txt")
print_word_analogies_data(word_analogies_new)

Résultat de Pearson :
  Statistique : 0.4789922671821398
  Valeur P : 1.2016493237138324e-21

Résultat de Significativité (Spearman) :
  Statistique : 0.487809233175005
  Valeur P : 1.6756599211095256e-22

Ensuite voici les résultat sur les analogies :
Nombre d'analogies : 200
Nombre d'analogies correctes : 92
Nombre d'analogies incorrectes : 108
Précision : 0.46


Alors là c'est un peu l'histoire opposé du modèle précédent. Le modèle n'obtient pas de bon résultat sur les corrélations de Pearson et Spearman (0.48 et 0.49 respectivement), mais il est un peu meilleur que le modèle précédent sur les analogies (0.46 contre 0.32).

Je pense les résultats mauvais de Pearson et Spearman viennent du fait que le modèle est super petit (3.71 Mo).